### Let's try the OpenAI API

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()
OPENAI_KEY = os.getenv('OPENAI_API_KEY')


In [2]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_KEY)

result = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What is 2+2",
        }
    ],
    model="gpt-4o",
)

print(result.choices[0].message.content)

2 + 2 equals 4.


## E-Commerce Review Processor
We will create a function that takes a review and returns a json object with the following fields:
- complaint: a boolean indicating if the review is a complaint
- Department: the appropriate department to handle the complaint. Options are Health, Food, Toy, Tool, Apparel, and Other.

In [ ]:
review = "These pants are not as good as I thought. They fabric is too rough."

system_prompt = """
You are an assistant that processes e-commerce reviews and returns a json object with the following fields:
    complaint: a Yes or No value indicating if the review is a complaint, 
    Department: the appropriate department to handle the complaint. Options are Health, Food, Toy, Tool, Apparel, gardening, cosmetics and Other. 

See the following examples for the format of the json object: 
Review: I love my new shirt! \n\n
{
    'complaint': 'No',
    'Department': 'Apparel'
}
Review: The walnuts were rotten and I had to return it. 
{
    'complaint': 'Yes',
    'Department': 'Food'
}
"""

result = client.chat.completions.create(
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Review: {review}"}
    ],
    model="gpt-4o",
)
print(result.choices[0].message.content)

{
    'complaint': 'Yes',
    'Department': 'Apparel'
}


In [3]:
review = "These pants are not as good as I thought. They fabric is too rough."

system_prompt = """
Indicate if the review is complaint or not and which department it belongs too. The options are Food, Apparel, Cosmetics, and Toys.

"""

result = client.chat.completions.create(
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Review: {review}"}
    ],
    model="gpt-4o",
)
print(result.choices[0].message.content)

Complaint, Apparel


In [19]:
review = "These pants are not as good as I thought. They fabric is too rough."

system_prompt = """
You are an assistant that processes e-commerce reviews and returns a json object with the following fields:
    complaint: a Yes or No value indicating if the review is a complaint, 
    Department: the appropriate department to handle the complaint. Options are Health, Food, Toy, Tool, Apparel, gardening and Other. 

"""

result = client.chat.completions.create(
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Review: {review}"}
    ],
    model="gpt-4o",
)
print(result.choices[0].message.content)

```json
{
    "complaint": "Yes",
    "Department": "Apparel"
}
```


In [7]:
review = "These pants are not as good as I thought. They fabric is too rough."

system_prompt = """
You are an assistant that processes e-commerce reviews and returns a json object with the following fields:
    complaint: a Yes or No value indicating if the review is a complaint, 
    Department: the appropriate department to handle the complaint. Options are Health, Food, Toy, Tool, Apparel and Other. 

See the following examples for the format of the json object: 
Review: I love my new shirt! \n\n
{
    'complaint': 'No',
    'Department': 'Apparel'
}
Review: The walnuts were rotten and I had to return it. 
{
    'complaint': 'Yes',
    'Department': 'Food'
}
"""

result = client.chat.completions.create(
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Review: {review}"}
    ],
    model="gpt-4o",
)
print(result.choices[0].message.content)

{
    'complaint': 'Yes',
    'Department': 'Apparel'
}


In [12]:
print(system_prompt)


You are an assistant that processes e-commerce reviews and returns a json object with the following fields:
    complaint: a Yes or No value indicating if the review is a complaint, 
    Department: the appropriate department to handle the complaint. Options are Health, Food, Toy, Tool, Apparel, and Other. 

See the following examples for the format of the json object: 
Review: I love my new shirt! 


{
    'complaint': 'No',
    'Department': 'Apparel'
}
Review: The walnuts were rotten and I had to return it. 
{
    'complaint': 'Yes',
    'Department': 'Food'
}



## Let's build this as a gradio app


In [16]:
import gradio as gr

def process_review(review):    
    result = client.chat.completions.create(
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Review: {review}"}
        ],
        model="gpt-4o",
    )
    return result.choices[0].message.content

iface = gr.Interface(
    fn=process_review,
    inputs=gr.Textbox(lines=3, placeholder="Enter your product review here..."),
    outputs=gr.Textbox(label="Analysis Result"),
    title="E-Commerce Review Analyzer",
    description="Enter a product review to analyze if it's a complaint and determine the relevant department."
)

# Launch the app
iface.launch()

/Users/amirhosseinmohajerani/Desktop/code/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/amirhosseinmohajerani/Desktop/code/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [ ]:
# Let's fine tune a model so we can ommit the system message during inference
# training data
# gpt-4o-mini-2024-07-18


In [22]:
import json

system_message = """You are an assistant that processes e-commerce reviews and returns a json object with the following fields: 
    complaint: a Yes or No value indicating if the review is a complaint, 
    Department: the appropriate department to handle the complaint. Options are Health, Food, Toy, Tool, Apparel, and Other."""

conversations = [
    # Apparel Department (10 examples)
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: The winter coat is perfect for cold weather and looks exactly like the picture!"},
            {"role": "assistant", "content": "{\n    'complaint': 'No',\n    'Department': 'Apparel'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: The jeans ripped after just two washes. Poor quality for the price."},
            {"role": "assistant", "content": "{\n    'complaint': 'Yes',\n    'Department': 'Apparel'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: These running shoes are incredibly comfortable and supportive."},
            {"role": "assistant", "content": "{\n    'complaint': 'No',\n    'Department': 'Apparel'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: The dress size runs way too small compared to the size chart."},
            {"role": "assistant", "content": "{\n    'complaint': 'Yes',\n    'Department': 'Apparel'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: Love these socks! They're so soft and durable."},
            {"role": "assistant", "content": "{\n    'complaint': 'No',\n    'Department': 'Apparel'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: The sweater started pilling after one wear. Very disappointing."},
            {"role": "assistant", "content": "{\n    'complaint': 'Yes',\n    'Department': 'Apparel'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: Perfect fit on these gloves and they keep my hands warm!"},
            {"role": "assistant", "content": "{\n    'complaint': 'No',\n    'Department': 'Apparel'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: The buttons fell off this shirt after first wash."},
            {"role": "assistant", "content": "{\n    'complaint': 'Yes',\n    'Department': 'Apparel'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: These yoga pants are perfect for my morning workout!"},
            {"role": "assistant", "content": "{\n    'complaint': 'No',\n    'Department': 'Apparel'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: The color of the hat is completely different from what was shown online."},
            {"role": "assistant", "content": "{\n    'complaint': 'Yes',\n    'Department': 'Apparel'\n}"}
        ]
    },
    
    # Food Department (10 examples)
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: These organic strawberries were moldy when they arrived."},
            {"role": "assistant", "content": "{\n    'complaint': 'Yes',\n    'Department': 'Food'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: The coffee beans are incredibly fresh and aromatic!"},
            {"role": "assistant", "content": "{\n    'complaint': 'No',\n    'Department': 'Food'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: The expiration date on the milk was already passed when delivered."},
            {"role": "assistant", "content": "{\n    'complaint': 'Yes',\n    'Department': 'Food'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: Best chocolate cookies I've ever tasted!"},
            {"role": "assistant", "content": "{\n    'complaint': 'No',\n    'Department': 'Food'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: The spice mix was stale and had no flavor."},
            {"role": "assistant", "content": "{\n    'complaint': 'Yes',\n    'Department': 'Food'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: Fresh and crispy apples, will order again!"},
            {"role": "assistant", "content": "{\n    'complaint': 'No',\n    'Department': 'Food'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: Found a foreign object in my cereal box."},
            {"role": "assistant", "content": "{\n    'complaint': 'Yes',\n    'Department': 'Food'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: The olive oil is authentic and tastes amazing!"},
            {"role": "assistant", "content": "{\n    'complaint': 'No',\n    'Department': 'Food'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: Package arrived with broken eggs and made a mess."},
            {"role": "assistant", "content": "{\n    'complaint': 'Yes',\n    'Department': 'Food'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: These protein bars are delicious and filling!"},
            {"role": "assistant", "content": "{\n    'complaint': 'No',\n    'Department': 'Food'\n}"}
        ]
    },

    # Health Department (10 examples)
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: These vitamins have improved my energy levels significantly!"},
            {"role": "assistant", "content": "{\n    'complaint': 'No',\n    'Department': 'Health'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: The probiotic caused severe stomach issues."},
            {"role": "assistant", "content": "{\n    'complaint': 'Yes',\n    'Department': 'Health'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: Great first aid kit, has everything needed!"},
            {"role": "assistant", "content": "{\n    'complaint': 'No',\n    'Department': 'Health'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: The supplement bottle was only half full when it arrived."},
            {"role": "assistant", "content": "{\n    'complaint': 'Yes',\n    'Department': 'Health'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: This natural sleep aid works wonders!"},
            {"role": "assistant", "content": "{\n    'complaint': 'No',\n    'Department': 'Health'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: The protein powder has a terrible artificial taste."},
            {"role": "assistant", "content": "{\n    'complaint': 'Yes',\n    'Department': 'Health'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: Love this natural toothpaste, leaves my mouth feeling fresh!"},
            {"role": "assistant", "content": "{\n    'complaint': 'No',\n    'Department': 'Health'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: The face cream caused a severe allergic reaction."},
            {"role": "assistant", "content": "{\n    'complaint': 'Yes',\n    'Department': 'Health'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: These compression socks really help with my circulation!"},
            {"role": "assistant", "content": "{\n    'complaint': 'No',\n    'Department': 'Health'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: The essential oil diffuser stopped working after a week."},
            {"role": "assistant", "content": "{\n    'complaint': 'Yes',\n    'Department': 'Health'\n}"}
        ]
    },

    # Tool Department (10 examples)
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: This screwdriver set is perfect for all my home repairs!"},
            {"role": "assistant", "content": "{\n    'complaint': 'No',\n    'Department': 'Tool'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: The power drill died after only two uses."},
            {"role": "assistant", "content": "{\n    'complaint': 'Yes',\n    'Department': 'Tool'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: Great quality hammer, very durable!"},
            {"role": "assistant", "content": "{\n    'complaint': 'No',\n    'Department': 'Tool'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: The measuring tape shows incorrect measurements."},
            {"role": "assistant", "content": "{\n    'complaint': 'Yes',\n    'Department': 'Tool'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: This toolbox is sturdy and well-organized!"},
            {"role": "assistant", "content": "{\n    'complaint': 'No',\n    'Department': 'Tool'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: The saw blade was dull right out of the package."},
            {"role": "assistant", "content": "{\n    'complaint': 'Yes',\n    'Department': 'Tool'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: These work gloves provide excellent protection!"},
            {"role": "assistant", "content": "{\n    'complaint': 'No',\n    'Department': 'Tool'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: The ladder's locking mechanism is faulty and unsafe."},
            {"role": "assistant", "content": "{\n    'complaint': 'Yes',\n    'Department': 'Tool'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: Perfect wrench set for my automotive work!"},
            {"role": "assistant", "content": "{\n    'complaint': 'No',\n    'Department': 'Tool'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: The electric sander stopped working mid-project."},
            {"role": "assistant", "content": "{\n    'complaint': 'Yes',\n    'Department': 'Tool'\n}"}
        ]
    },

    # Toy Department (10 examples)
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: This educational toy keeps my kids engaged for hours!"},
            {"role": "assistant", "content": "{\n    'complaint': 'No',\n    'Department': 'Toy'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: The remote control car broke on the first day."},
            {"role": "assistant", "content": "{\n    'complaint': 'Yes',\n    'Department': 'Toy'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: Great quality board game, whole family loves it!"},
            {"role": "assistant", "content": "{\n    'complaint': 'No',\n    'Department': 'Toy'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: The puzzle is missing several pieces."},
            {"role": "assistant", "content": "{\n    'complaint': 'Yes',\n    'Department': 'Toy'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: These building blocks are perfect for my toddler!"},
            {"role": "assistant", "content": "{\n    'complaint': 'No',\n    'Department': 'Toy'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: The stuffed animal fell apart at the seams immediately."},
            {"role": "assistant", "content": "{\n    'complaint': 'Yes',\n    'Department': 'Toy'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: This science kit is both fun and educational!"},
            {"role": "assistant", "content": "{\n    'complaint': 'No',\n    'Department': 'Toy'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: The art set arrived with dried-out markers."},
            {"role": "assistant", "content": "{\n    'complaint': 'Yes',\n    'Department': 'Toy'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: My daughter loves this dollhouse!"},
            {"role": "assistant", "content": "{\n    'complaint': 'No',\n    'Department': 'Toy'\n}"}
        ]
    },
    {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": "Review: The toy guitar's strings broke within an hour."},
            {"role": "assistant", "content": "{\n    'complaint': 'Yes',\n    'Department': 'Toy'\n}"}
        ]
    }
]

# Write to JSONL file
with open('ecommerce_reviews_training.jsonl', 'w') as f:
    for conversation in conversations:
        f.write(json.dumps(conversation) + '\n')

In [4]:
client.files.create(
  file=open("ecommerce_reviews_training.jsonl", "rb"),
  purpose="fine-tune"
)


FileObject(id='file-Bq9cUCyu3Wbg9Xh3TCnwmi', bytes=26625, created_at=1736602241, filename='ecommerce_reviews_training.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [6]:
file_id = 'file-Bq9cUCyu3Wbg9Xh3TCnwmi'

client.fine_tuning.jobs.create(
  training_file=file_id,
  model="gpt-4o-mini-2024-07-18",
  suffix="ecommerce-reviews-ft"
)
# id=ftjob-jUBrCZ8lOA1UX7HA1DuO84pl

FineTuningJob(id='ftjob-jUBrCZ8lOA1UX7HA1DuO84pl', created_at=1736602333, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-A8knzBFdDBhKJEnbNAeJtNgx', result_files=[], seed=72038912, status='validating_files', trained_tokens=None, training_file='file-Bq9cUCyu3Wbg9Xh3TCnwmi', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto')), type='supervised'), user_provided_suffix='ecommerce-reviews-ft')

In [12]:
organization_id='org-A8knzBFdDBhKJEnbNAeJtNgx'
# let's check the status of the job
client.fine_tuning.jobs.retrieve("ftjob-jUBrCZ8lOA1UX7HA1DuO84pl")

FineTuningJob(id='ftjob-jUBrCZ8lOA1UX7HA1DuO84pl', created_at=1736602333, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal:ecommerce-reviews-ft:AoVzUWhf', finished_at=1736602734, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier=1.8, n_epochs=3), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-A8knzBFdDBhKJEnbNAeJtNgx', result_files=['file-V2Ds6PS7Pn2uLKn5Hccs44'], seed=72038912, status='succeeded', trained_tokens=15129, training_file='file-Bq9cUCyu3Wbg9Xh3TCnwmi', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size=1, learning_rate_multiplier=1.8, n_epochs=3)), type='supervised'), user_provided_suffix='ecommerce-reviews-ft')

In [6]:
# Let's test the model
review='I love my new shirt!'
result = client.chat.completions.create(
    messages=[
        {"role": "system", "content": 'You are an assistant that processes e-commerce reviews and returns a json object with the following fields: \n    complaint: a Yes or No value indicating if the review is a complaint, \n    Department: the appropriate department to handle the complaint. Options are Health, Food, Toy, Tool, Apparel, and Other.'},
        {"role": "user", "content": f"Review: {review}"}
    ],
    model="ft:gpt-4o-mini-2024-07-18:personal:ecommerce-reviews-ft:AoVzUWhf",
)
print(result.choices[0].message.content)

{
    'complaint': 'No',
    'Department': 'Apparel'
}
